# 《Reinforcement Learning》

![](https://github.com/applenob/rl_learn/raw/master/res/rl_book.jpg)

## 目录

- [0.书本介绍](#0.书本介绍)
- [1. Introduction](#1.-Introduction)
- [2. Multi-armed Bandits](#2.-Multi-armed-Bandits)
- [3. Finite Markov DecisionProcesses](#3.-Finite-Markov-DecisionProcesses)
- [4. Dynamic Programming](#4.-Dynamic-Programming)
- [5. Monte Carlo Methods](#5.-Monte-Carlo-Methods)
- [](#)
- [](#)
- [](#)
- [](#)
- [](#)
- [](#)
- [](#)
- [](#)
- [](#)
- [](#)
- [](#)
- [](#)
- [](#)

## 0.书本介绍

[Reinforcement Learning](https://book.douban.com/subject/2866455/)

作者: Richard S. Sutton / Andrew G. Barto 

内容简介：

```
Reinforcement learning, one of the most active research areas in artificial intelligence, is a computational approach to learning whereby an agent tries to maximize the total amount of reward it receives when interacting with a complex, uncertain environment. 

In Reinforcement Learning, Richard Sutton and Andrew Barto provide a clear and simple account of the key ideas and algorithms of reinforcement learning. Their discussion ranges from the history of the field's intellectual foundations to the most recent developments and applications. 

The only necessary mathematical background is familiarity with elementary concepts of probability.

The book is divided into three parts. 

Part I defines the reinforcement learning problem in terms of Markov decision processes. 

Part II provides basic solution methods: dynamic programming, Monte Carlo methods, and temporal-difference learning. 

Part III presents a unified view of the solution methods and incorporates artificial neural networks, eligibility traces, and planning; the two final chapters present case studies and consider the future of reinforcement learning.
```

其他重要学习资料：

- [WildML的博客](http://www.wildml.com/2016/10/learning-reinforcement-learning/)

- [David Silver’s Reinforcement Learning Course](http://www0.cs.ucl.ac.uk/staff/d.silver/web/Teaching.html)


## 1. Introduction

**Reinforcement Learning** :强化学习是学习“该如何去做”，如何从某个状态映射到某个行为，来最大化某个数值的奖励信号。

**Exploit vs Explore**: **exploit** 代表利用已有的信息去获得最大的奖励，**explore** 代表去探索一些没有尝试过的行为，去获得可能的更大的奖励。

**强化学习的几个要素**:
- **policy**: 状态到行为的映射，定义agent在某个状态下应该如何采取行为。
- **reward function**: 在某个状态下，agent能够收到的即时反馈。
- **value function**: 衡量在某个状态下，能够获得的长期反馈。
- **modle (of the environment)**: 模型用来模拟环境的行为，给定一个状态和动作，模型可以预测下一个状态和奖励。
   
**RL vs Evolutionary Methods**:
- Evolutionary Methods（遗传算法，具体可以回顾[之前的博客](https://applenob.github.io/ga.html)），直接在policy空间中搜索，并计算最后的得分。通过一代代的进化来找寻最优policy。
- 遗传算法忽略了policy实际上是state到action的映射，它不关注agent和环境的互动，只看最终结果。

**Tic-Tac-Toe（井字棋）**：
![](https://github.com/applenob/rl_learn/raw/master/res/ttt.png)
- 定义policy是任何一种局面下，该如何落子。
- 遗传算法解法：试很多种policy，找到最终胜利的几种，然后结合，更新。
- 强化学习解法：
    - 1.建立一张表格，state_num × 1，代表每个state下，获胜的概率，这个表格就是所谓的**value function**。
    - 2.跟对手下很多局。每次落子的时候，依据是在某个state下，选择所有可能的后继state中，获胜概率最大的（v最大的）。这种方法即贪婪法（Exploit）。偶尔我们也随机选择一些其他的state（Explore）。
    - 3.“back up”后继state的v到当前state上。$V(s)\leftarrow V(s)+\alpha[V(s')-V(s)]$，这就是所谓的**差分学习**，temporal-difference learning，这么叫是因为$V(s')-V(s)$是两个时间点上的两次估计的差。

## 2. Multi-armed Bandits

**k-armed Bandit Problem**：
![](https://github.com/applenob/rl_learn/raw/master/res/bandit.png)
- 问题描述：重复地面临有k中选择的情况，每一次选择之后，都会收到一个reward。这个reward服从一个跟你的选择相关的分布。你的目标是，在选择t次后，找到最大的期望reward。
- 形式化定义：假设时刻$t$的动作是$A_t$，reward是$R_t$，定义价值函数：$q_*(a) = E[R_t|A_t=a]$。
- 特点：每个时刻下，$q_*(a)$是固定的，stationary。
- 如果我们知道每个动作a对应的价值q，那么这个问题对我们来说就已经解决了，即，我们选择q最大的那个a。
- 然而，我们并不知道实际上的q是什么，只能估计$Q_t(a)\approx q_*(a)$。
- 在每个时间节点上，如果总是选择对应的估计Q最大的a，这被称为**greedy**。
- **$\epsilon$-greedy**：*：$1-\epsilon$的概率选择Q最大的a，*$\epsilon$的概率选择其他a。
- 估计$Q_t(a) = \frac{在时刻t之前，所有采用动作a获得的奖励之和}{在时刻t之前，采用动作a的次数} = \frac{\sum^{t-1}_{i=1}R_i\cdot\mathbf{1}_{A_i=a}}{\sum^{t-1}_{i=1}\mathbf{1}_{A_i=a}}$，被称为sample-average。

**10-armed Testbed**：
- 每次动作的选择，从0~10，服从标准高斯分布（均值是0，方差是1）。
- $R_t$服从高斯分布，均值是$q_*(A_t)$，方差是1。
- 可以规定每次实验做1000次选择，称为一个run。
- 一共跑了2000个独立的run。
- 实验结果（**$\epsilon$-greedy**的优越性）：
![](https://github.com/applenob/rl_learn/raw/master/res/average_reward.png)

**增量式的估计**：
- 上面的sample-average即：$Q_n = \frac{R_1+R_2+...+R_{n-1}}{n-1}$
- 改写成增量式的形式：$Q_{n+1} = Q_n + \frac{1}{n}[R_n-Q_n]$
- 即：新估计 = 老估计 + 步长 × [奖励 - 老估计]
![](https://github.com/applenob/rl_learn/raw/master/res/bandit_algo.png)

**Optimistic Initial Values**：
- 设置初始的$Q_1(a)$为一些较大的值。
- 这样会鼓励explore。
- 对于nonstationary的问题不适用。
![](https://github.com/applenob/rl_learn/raw/master/res/)

**Upper-Confidence-Bound Action Selection（UCB）**：
- $A_t = \underset{a}{argmax}[Q_t(a)+c\sqrt{\frac{log\;t}{N_t(a)}}]$
- 其中，$N_t(a)$代表动作a在t之前出现的次数，根号项衡量动作a的不确定性，如果某个动作已经使用了很多次，则倾向使用使用次数少的，这样达到explore的效果。
![](https://github.com/applenob/rl_learn/raw/master/res/)

**Gradient Bandit Algorithms**：
- 使用一个数值表示对某个动作的**偏好**：$H_t(a)$
- $Pr\{A_t=a\}=\frac{e^{H_t(a)}}{\sum_{b=1}^k{e^{H_t(b)}}}=\pi_t(a)$
- 更新规则（不必细究，学到policy gradient自然明白）：
    - $H_{t+1}(A_t) = H_t(A_t) + \alpha(R_t-\overline R_t)(1-\pi _t(A_t))$
    - $\forall a \neq A_t$
![](https://github.com/applenob/rl_learn/raw/master/res/)

**各种方法对比**：
![](https://github.com/applenob/rl_learn/raw/master/res/)

## 3. Finite Markov DecisionProcesses

****：

****：

****：

****：

****：

****：

****：

****：

****：

****：

## 4. Dynamic Programming

****：

****：

****：

****：

****：

****：

****：

****：

****：

****：

## 5. Monte Carlo Methods

****：

****：

****：

****：

****：

****：

****：

****：

****：

****：

In [1]:
print "done"

done
